In [1]:
import numpy as np
import math
import gym
from gym import spaces
import time
import random
from stable_baselines3 import A2C
import torch
import matplotlib.pyplot as plt
import heapq
from stable_baselines3.common.env_checker import check_env
from mpl_toolkits.mplot3d import Axes3D

In [2]:
torch.__version__

'1.13.1'

In [3]:
gym.__version__

'0.21.0'

In [4]:
%matplotlib inline

## Finite Element Model of the Space Frame Element

In [5]:
def PlaneTrussElementLength(x1,y1,z1,x2,y2,z2):
    return math.sqrt((x2-x1)*(x2-x1) + (y2-y1)*(y2-y1)+(z2-z1)*(z2-z1))

In [6]:
def SpaceFrameElementStiffness(E,G,A,Iy,Iz,J,x1,y1,z1,x2,y2,z2):
    L = PlaneTrussElementLength(x1,y1,z1,x2,y2,z2)
    w1 = E*A/L
    w2 = 12*E*Iz/(L*L*L)
    w3 = 6*E*Iz/(L*L)
    w4 = 4*E*Iz/L
    w5 = 2*E*Iz/L
    w6 = 12*E*Iy/(L*L*L)
    w7 = 6*E*Iy/(L*L)
    w8 = 4*E*Iy/L
    w9 = 2*E*Iy/L
    w10 = G*J/L
    
    kprime = np.array([[w1, 0, 0, 0, 0, 0, -w1, 0, 0, 0, 0, 0],
                        [0, w2, 0, 0, 0, w3, 0, -w2, 0, 0, 0, w3], 
                        [0, 0, w6, 0, -w7, 0, 0, 0, -w6, 0, -w7, 0],
                        [0, 0, 0, w10, 0, 0, 0, 0, 0, -w10, 0, 0],
                        [0, 0, -w7, 0, w8, 0, 0, 0, w7, 0, w9, 0],
                        [0, w3, 0, 0, 0, w4, 0, -w3, 0, 0, 0, w5],
                        [-w1, 0, 0, 0, 0, 0, w1, 0, 0, 0, 0, 0],
                        [0, -w2, 0, 0, 0, -w3, 0, w2, 0, 0, 0, -w3],
                        [0, 0, -w6, 0, w7, 0, 0, 0, w6, 0, w7, 0],
                        [0, 0, 0, -w10, 0, 0, 0, 0, 0, w10, 0, 0],
                        [0, 0, -w7, 0, w9, 0, 0, 0, w7, 0, w8, 0],
                        [0, w3, 0, 0, 0, w5, 0, -w3, 0, 0, 0, w4]])  
    
    
    if x1 == x2 and y1 == y2:
        if z2 > z1:
            Lambda = np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]])
        else:
            Lambda = np.array([[0, 0, -1], [0, 1, 0], [1, 0, 0]])
    else:
        CXx = (x2-x1)/L
        CYx = (y2-y1)/L
        CZx = (z2-z1)/L
        D = math.sqrt(CXx*CXx + CYx*CYx)
        CXy = -CYx/D
        CYy = CXx/D
        CZy = 0
        CXz = -CXx*CZx/D
        CYz = -CYx*CZx/D
        CZz = D
        Lambda = np.array([[CXx, CYx, CZx], [CXy, CYy, CZy], [CXz, CYz, CZz]])
        
        
    R = np.array([np.concatenate((np.concatenate((Lambda,np.zeros((3,3)),np.zeros((3,3)),np.zeros((3,3))),axis=1),
        np.concatenate((np.zeros((3,3)), Lambda, np.zeros((3,3)), np.zeros((3,3))),axis=1) ,
        np.concatenate((np.zeros((3,3)), np.zeros((3,3)), Lambda, np.zeros((3,3))),axis=1), 
        np.concatenate((np.zeros((3,3)), np.zeros((3,3)), np.zeros((3,3)), Lambda),axis=1)))])[0]
    return np.dot(np.dot(R.T,kprime),R)           

In [7]:
def SpaceFrameAssemble(K,k,i,j):
    K[6*i,6*i] = K[6*i,6*i] + k[0,0]
    K[6*i,6*i+1] = K[6*i,6*i+1] + k[0,1]
    K[6*i,6*i+2] = K[6*i,6*i+2] + k[0,2]
    K[6*i,6*i+3] = K[6*i,6*i+3] + k[0,3]
    K[6*i,6*i+4] = K[6*i,6*i+4] + k[0,4]
    K[6*i,6*i+5] = K[6*i,6*i+5] + k[0,5]
    K[6*i,6*j] = K[6*i,6*j] + k[0,6]
    K[6*i,6*j+1] = K[6*i,6*j+1] + k[0,7]
    K[6*i,6*j+2] = K[6*i,6*j+2] + k[0,8]
    K[6*i,6*j+3] = K[6*i,6*j+3] + k[0,9]
    K[6*i,6*j+4] = K[6*i,6*j+4] + k[0,10]
    K[6*i,6*j+5] = K[6*i,6*j+5] + k[0,11]
    K[6*i+1,6*i] = K[6*i+1,6*i] + k[1,0]
    K[6*i+1,6*i+1] = K[6*i+1,6*i+1] + k[1,1]
    K[6*i+1,6*i+2] = K[6*i+1,6*i+2] + k[1,2]
    K[6*i+1,6*i+3] = K[6*i+1,6*i+3] + k[1,3]
    K[6*i+1,6*i+4] = K[6*i+1,6*i+4] + k[1,4]
    K[6*i+1,6*i+5] = K[6*i+1,6*i+5] + k[1,5]
    K[6*i+1,6*j] = K[6*i+1,6*j] + k[1,6]
    K[6*i+1,6*j+1] = K[6*i+1,6*j+1] + k[1,7]
    K[6*i+1,6*j+2] = K[6*i+1,6*j+2] + k[1,8]
    K[6*i+1,6*j+3] = K[6*i+1,6*j+3] + k[1,9]
    K[6*i+1,6*j+4] = K[6*i+1,6*j+4] + k[1,10]
    K[6*i+1,6*j+5] = K[6*i+1,6*j+5] + k[1,11]
    K[6*i+2,6*i]   = K[6*i+2,6*i] + k[2,0]
    K[6*i+2,6*i+1] = K[6*i+2,6*i+1] + k[2,1]
    K[6*i+2,6*i+2] = K[6*i+2,6*i+2] + k[2,2]
    K[6*i+2,6*i+3] = K[6*i+2,6*i+3] + k[2,3]
    K[6*i+2,6*i+4] = K[6*i+2,6*i+4] + k[2,4]
    K[6*i+2,6*i+5] = K[6*i+2,6*i+5] + k[2,5]
    K[6*i+2,6*j]   = K[6*i+2,6*j] + k[2,6]
    K[6*i+2,6*j+1] = K[6*i+2,6*j+1] + k[2,7]
    K[6*i+2,6*j+2] = K[6*i+2,6*j+2] + k[2,8]
    K[6*i+2,6*j+3] = K[6*i+2,6*j+3] + k[2,9]
    K[6*i+2,6*j+4] = K[6*i+2,6*j+4] + k[2,10]
    K[6*i+2,6*j+5] = K[6*i+2,6*j+5] + k[2,11]
    K[6*i+3,6*i] = K[6*i+3,6*i] + k[3,0]
    K[6*i+3,6*i+1] = K[6*i+3,6*i+1] + k[3,1]
    K[6*i+3,6*i+2] = K[6*i+3,6*i+2] + k[3,2]
    K[6*i+3,6*i+3] = K[6*i+3,6*i+3] + k[3,3]
    K[6*i+3,6*i+4] = K[6*i+3,6*i+4] + k[3,4]
    K[6*i+3,6*i+5] = K[6*i+3,6*i+5] + k[3,5]
    K[6*i+3,6*j] = K[6*i+3,6*j] + k[3,6]
    K[6*i+3,6*j+1] = K[6*i+3,6*j+1] + k[3,7]
    K[6*i+3,6*j+2] = K[6*i+3,6*j+2] + k[3,8]    
    K[6*i+3,6*j+3] = K[6*i+3,6*j+3] + k[3,9]
    K[6*i+3,6*j+4] = K[6*i+3,6*j+4] + k[3,10]
    K[6*i+3,6*j+5] = K[6*i+3,6*j+5] + k[3,11]
    K[6*i+4,6*i] = K[6*i+4,6*i] + k[4,0]
    K[6*i+4,6*i+1] = K[6*i+4,6*i+1] + k[4,1]
    K[6*i+4,6*i+2] = K[6*i+4,6*i+2] + k[4,2]
    K[6*i+4,6*i+3] = K[6*i+4,6*i+3] + k[4,3]
    K[6*i+4,6*i+4] = K[6*i+4,6*i+4] + k[4,4]
    K[6*i+4,6*i+5] = K[6*i+4,6*i+5] + k[4,5]
    K[6*i+4,6*j] = K[6*i+4,6*j] + k[4,6]
    K[6*i+4,6*j+1] = K[6*i+4,6*j+1] + k[4,7]
    K[6*i+4,6*j+2] = K[6*i+4,6*j+2] + k[4,8]
    K[6*i+4,6*j+3] = K[6*i+4,6*j+3] + k[4,9]
    K[6*i+4,6*j+4] = K[6*i+4,6*j+4] + k[4,10]
    K[6*i+4,6*j+5] = K[6*i+4,6*j+5] + k[4,11]
    K[6*i+5,6*i] = K[6*i+5,6*i] + k[5,0]
    K[6*i+5,6*i+1] = K[6*i+5,6*i+1] + k[5,1]
    K[6*i+5,6*i+2] = K[6*i+5,6*i+2] + k[5,2]
    K[6*i+5,6*i+3] = K[6*i+5,6*i+3] + k[5,3]
    K[6*i+5,6*i+4] = K[6*i+5,6*i+4] + k[5,4]
    K[6*i+5,6*i+5] = K[6*i+5,6*i+5] + k[5,5]
    K[6*i+5,6*j] = K[6*i+5,6*j] + k[5,6]
    K[6*i+5,6*j+1] = K[6*i+5,6*j+1] + k[5,7]
    K[6*i+5,6*j+2] = K[6*i+5,6*j+2] + k[5,8]
    K[6*i+5,6*j+3] = K[6*i+5,6*j+3] + k[5,9]
    K[6*i+5,6*j+4] = K[6*i+5,6*j+4] + k[5,10]
    K[6*i+5,6*j+5] = K[6*i+5,6*j+5] + k[5,11]
    K[6*j,6*i] = K[6*j,6*i] + k[6,0]
    K[6*j,6*i+1] = K[6*j,6*i+1] + k[6,1]
    K[6*j,6*i+2] = K[6*j,6*i+2] + k[6,2]
    K[6*j,6*i+3] = K[6*j,6*i+3] + k[6,3]
    K[6*j,6*i+4] = K[6*j,6*i+4] + k[6,4]
    K[6*j,6*i+5] = K[6*j,6*i+5] + k[6,5]
    K[6*j,6*j] = K[6*j,6*j] + k[6,6]
    K[6*j,6*j+1] = K[6*j,6*j+1] + k[6,7]
    K[6*j,6*j+2] = K[6*j,6*j+2] + k[6,8]
    K[6*j,6*j+3] = K[6*j,6*j+3] + k[6,9]
    K[6*j,6*j+4] = K[6*j,6*j+4] + k[6,10]
    K[6*j,6*j+5] = K[6*j,6*j+5] + k[6,11]
    K[6*j+1,6*i] = K[6*j+1,6*i] + k[7,0]
    K[6*j+1,6*i+1] = K[6*j+1,6*i+1] + k[7,1]
    K[6*j+1,6*i+2] = K[6*j+1,6*i+2] + k[7,2]
    K[6*j+1,6*i+3] = K[6*j+1,6*i+3] + k[7,3]
    K[6*j+1,6*i+4] = K[6*j+1,6*i+4] + k[7,4]
    K[6*j+1,6*i+5] = K[6*j+1,6*i+5] + k[7,5]
    K[6*j+1,6*j] = K[6*j+1,6*j] + k[7,6]
    K[6*j+1,6*j+1] = K[6*j+1,6*j+1] + k[7,7]
    K[6*j+1,6*j+2] = K[6*j+1,6*j+2] + k[7,8]
    K[6*j+1,6*j+3] = K[6*j+1,6*j+3] + k[7,9]
    K[6*j+1,6*j+4] = K[6*j+1,6*j+4] + k[7,10]
    K[6*j+1,6*j+5] = K[6*j+1,6*j+5] + k[7,11]
    K[6*j+2,6*i] = K[6*j+2,6*i] + k[8,0]
    K[6*j+2,6*i+1] = K[6*j+2,6*i+1] + k[8,1]
    K[6*j+2,6*i+2] = K[6*j+2,6*i+2] + k[8,2]
    K[6*j+2,6*i+3] = K[6*j+2,6*i+3] + k[8,3]
    K[6*j+2,6*i+4] = K[6*j+2,6*i+4] + k[8,4]
    K[6*j+2,6*i+5] = K[6*j+2,6*i+5] + k[8,5]
    K[6*j+2,6*j] = K[6*j+2,6*j] + k[8,6]
    K[6*j+2,6*j+1] = K[6*j+2,6*j+1] + k[8,7]
    K[6*j+2,6*j+2] = K[6*j+2,6*j+2] + k[8,8]
    K[6*j+2,6*j+3] = K[6*j+2,6*j+3] + k[8,9]
    K[6*j+2,6*j+4] = K[6*j+2,6*j+4] + k[8,10]
    K[6*j+2,6*j+5] = K[6*j+2,6*j+5] + k[8,11]
    K[6*j+3,6*i] = K[6*j+3,6*i] + k[9,0]
    K[6*j+3,6*i+1] = K[6*j+3,6*i+1] + k[9,1]
    K[6*j+3,6*i+2] = K[6*j+3,6*i+2] + k[9,2]
    K[6*j+3,6*i+3] = K[6*j+3,6*i+3] + k[9,3]
    K[6*j+3,6*i+4] = K[6*j+3,6*i+4] + k[9,4]
    K[6*j+3,6*i+5] = K[6*j+3,6*i+5] + k[9,5]
    K[6*j+3,6*j] = K[6*j+3,6*j] + k[9,6]
    K[6*j+3,6*j+1] = K[6*j+3,6*j+1] + k[9,7]
    K[6*j+3,6*j+2] = K[6*j+3,6*j+2] + k[9,8]
    K[6*j+3,6*j+3] = K[6*j+3,6*j+3] + k[9,9]
    K[6*j+3,6*j+4] = K[6*j+3,6*j+4] + k[9,10]
    K[6*j+3,6*j+5] = K[6*j+3,6*j+5] + k[9,11]
    K[6*j+4,6*i] = K[6*j+4,6*i] + k[10,0]
    K[6*j+4,6*i+1] = K[6*j+4,6*i+1] + k[10,1]
    K[6*j+4,6*i+2] = K[6*j+4,6*i+2] + k[10,2]
    K[6*j+4,6*i+3] = K[6*j+4,6*i+3] + k[10,3]
    K[6*j+4,6*i+4] = K[6*j+4,6*i+4] + k[10,4]
    K[6*j+4,6*i+5] = K[6*j+4,6*i+5] + k[10,5]
    K[6*j+4,6*j] = K[6*j+4,6*j] + k[10,6]
    K[6*j+4,6*j+1] = K[6*j+4,6*j+1] + k[10,7]
    K[6*j+4,6*j+2] = K[6*j+4,6*j+2] + k[10,8]
    K[6*j+4,6*j+3] = K[6*j+4,6*j+3] + k[10,9]
    K[6*j+4,6*j+4] = K[6*j+4,6*j+4] + k[10,10]
    K[6*j+4,6*j+5] = K[6*j+4,6*j+5] + k[10,11]
    K[6*j+5,6*i] = K[6*j+5,6*i] + k[11,0]
    K[6*j+5,6*i+1] = K[6*j+5,6*i+1] + k[11,1]
    K[6*j+5,6*i+2] = K[6*j+5,6*i+2] + k[11,2]
    K[6*j+5,6*i+3] = K[6*j+5,6*i+3] + k[11,3]
    K[6*j+5,6*i+4] = K[6*j+5,6*i+4] + k[11,4]
    K[6*j+5,6*i+5] = K[6*j+5,6*i+5] + k[11,5]
    K[6*j+5,6*j] = K[6*j+5,6*j] + k[11,6]
    K[6*j+5,6*j+1] = K[6*j+5,6*j+1] + k[11,7]
    K[6*j+5,6*j+2] = K[6*j+5,6*j+2] + k[11,8]
    K[6*j+5,6*j+3] = K[6*j+5,6*j+3] + k[11,9]
    K[6*j+5,6*j+4] = K[6*j+5,6*j+4] + k[11,10]
    K[6*j+5,6*j+5] = K[6*j+5,6*j+5] + k[11,11]
    
    return K

In [8]:
def FEA_u(coord, elcon, bc_node, bc_val, global_force, 
          E=210e6, G=84e6, A=2e-2, Iy=10e-5, Iz=20e-5, J=5e-5):
    coord=np.array(coord)
    elcon=np.array(elcon)
    
    K=np.zeros(shape=(6*(np.max(elcon)+1),6*(np.max(elcon)+1)))
    for el in elcon:
        k=SpaceFrameElementStiffness(E,G,A,Iy,Iz,J,
                                     coord[el[0]][0],coord[el[0]][1],coord[el[0]][2],\
                                     coord[el[1]][0],coord[el[1]][1],coord[el[1]][2])
        K=SpaceFrameAssemble(K,k,el[0],el[1])
        
    F = np.array(global_force)
    
    
    # https://github.com/CALFEM/calfem-matlab/blob/master/fem/solveq.m
    
    bc=np.array([bc_node, 
                bc_val]).T
    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))

    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.lstsq(K[fdof,:][:,fdof], (F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T, rcond=None)[0] 
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    
#     Q=np.dot(K,d).T-F 
    return d

In [9]:
# 10 The Space Frame Element - verification
d=FEA_u(np.array([0,0,0,
                  3,0,0,
                  0,0,-3,
                  0,-4,0]).reshape(4,3),
        elcon=np.array([[0, 1],
                      [0, 2],
                      [0, 3]]),
        bc_node=list(range(6,24)), 
        bc_val=[0]*18,
        global_force=[-10,0,20,0,0,0,
                                0,0,0,0,0,0,
                                0,0,0,0,0,0,
                                0,0,0,0,0,0])

In [10]:
d

array([-7.05147750e-06, -6.65367100e-08,  1.41769582e-05,  1.44778793e-06,
        1.74858422e-06,  1.13605431e-06,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

## Utils

In [11]:
def total_length(coord,elcon):
    coord=np.array(coord)
    elcon=np.array(elcon)
    t_length=0
    for i in range(len(elcon)):
        l=PlaneTrussElementLength(coord[elcon[i][0]][0],\
                                    coord[elcon[i][0]][1],\
                                    coord[elcon[i][0]][2],\
                                    coord[elcon[i][1]][0],\
                                    coord[elcon[i][1]][1],\
                                    coord[elcon[i][1]][2])
        t_length+=l        
    return t_length    

In [12]:
def possible_lines_dic(n,m,dx,dy):
    A=[(-dx,0),(-dx,dy),(0,dy),(dx,dy),(dx,0),(dx,-dy),(0,-dy),(-dx,-dy)]
    dic={}
    t=0
    for i in range(n):
        for j in range(m):
            for item in A:
                x,y=j*dx,i*dy
                x1,y1=x+item[0],y+item[1]
                if (x1>=0 and x1<=(m-1)*dx and 
                    y1>=0 and y1<=(n-1)*dy and 
                    (x1,y1,x,y) not in dic):
                    dic[(x,y,x1,y1)]=t
                    t+=1
    return dic                

In [13]:
DIM = len(possible_lines_dic(n=5,m=5,dx=1,dy=1)) + 3 # +2 for x and y +1 for action

## Model

In [14]:
class Model:
    def __init__(self,n=5,m=5,dx=1,dy=1, force=-500,
                 E=210e6, G=84e6, A=2e-2, Iy=10e-5, Iz=20e-5, J=5e-5, break_flag=False):
        # n,m,dx,dy - grid parameters    
        self.E=E
        self.G=G
        self.A=A
        self.Iy=Iy
        self.Iz=Iz
        self.J=J
        self.n=n
        self.m=m
        self.dx=dx
        self.dy=dy
        self.dic_lines=possible_lines_dic(self.n,self.m,self.dx,self.dy)
        self.line_list=len(self.dic_lines)*[0]
        self.break_flag=break_flag
        self.coord=[[2,2,0]]    
        self.elcon=[]  
        self.el_dic={(2,2):0}
        self.max_el=0
        self.force=force
        self.bc_node = []
        self.bc_val = []
        self.global_force = [0, self.force, 0, 0, 0, 0]
        self.old_weight=float("inf")
        self.old_strength=-float("inf")
        self.visit_list = [0,0,0,0] # number of checkpoints is 4
    
    def reset(self,break_flag,force):
        self.dic_lines=possible_lines_dic(self.n, self.m, self.dx, self.dy)
        self.line_list=len(self.dic_lines)*[0]
        self.break_flag=break_flag
        self.coord=[[2,2,0]]    
        self.elcon=[]  
        self.el_dic={(2,2):0}
        self.max_el=0
        self.force=force
        self.bc_node = []
        self.bc_val = []
        self.global_force = [0, self.force, 0, 0, 0, 0]
        self.visit_list = [0,0,0,0] # number of checkpoints is 4
    
    def FEA(self):
        return FEA_u(self.coord, 
                     self.elcon, 
                     self.bc_node, 
                     self.bc_val, 
                     self.global_force, )
        
    def max_u(self, FEA_output_arr):
        t=1
        A=[]
        while t<len(FEA_output_arr):
            A.append(FEA_output_arr[t])
            t+=6            
        return min(A)    
            
    
    def length(self):
        return total_length(self.coord,self.elcon)
    
    
    def move_w(self,x,y):
        # x,y - current location
        x_new=x-self.dx
        y_new=y
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
            
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])
            
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1
            
        return x_new, y_new
            
    def move_nw(self,x,y):
        # x,y - current location
        x_new=x-self.dx
        y_new=y+self.dy
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
            
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]]) 
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1    
                
        return x_new, y_new
    
    def move_n(self,x,y):
        # x,y - current location
        x_new=x
        y_new=y+self.dy
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
            
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])
                  
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1     
        
        return x_new, y_new
    
    
    def move_ne(self,x,y):
        # x,y - current location
        x_new=x+self.dx
        y_new=y+self.dy
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
                        
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1     
                  
        return x_new, y_new
    
    def move_e(self,x,y):
        # x,y - current location
        x_new=x+self.dx
        y_new=y
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
                       
                  
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])   
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1      
        return x_new, y_new
    
    def move_se(self,x,y):
        # x,y - current location
        x_new=x+self.dx
        y_new=y-self.dy
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
            
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1      
        
        return x_new, y_new
    
    def move_s(self,x,y):
        # x,y - current location
        x_new=x
        y_new=y-self.dy
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True 
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
            
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1      
        
        return x_new, y_new
    
    def move_sw(self,x,y):
        # x,y - current location
        x_new=x-self.dx
        y_new=y-self.dy
        if x_new<0 or x_new>(self.m-1)*self.dx or y_new<0 or y_new>(self.n-1)*self.dy \
        or 3 in self.line_list:
            self.break_flag=True
        else:
            try:
                self.line_list[self.dic_lines[(x,y,x_new,y_new)]]+=1
            except KeyError:
                self.line_list[self.dic_lines[(x_new,y_new, x,y)]]+=1
                        
            if (x_new,y_new) not in self.el_dic:
                self.max_el+=1
                self.el_dic[(x_new,y_new)]=self.max_el
                self.coord.append([x_new,y_new,0])
                if (x_new,y_new)!=(1,1) and (x_new,y_new)!=(self.m-2,1) and \
                    (x_new,y_new)!=(self.m-2,self.n-2) and \
                    (x_new,y_new)!=(1,self.n-2):
                    self.global_force += [0,self.force,0,0,0,0]
                elif (x_new,y_new)==(1,1) or (x_new,y_new)==(self.m-2,1) or \
                    (x_new,y_new)==(self.m-2,self.n-2) or \
                    (x_new,y_new)==(1,self.n-2):
                    range_ = list(range(self.el_dic[(x_new,y_new)]*6-6,self.el_dic[(x_new,y_new)]*6))
                    self.bc_node += range_
                    self.bc_val += [0]*len(range_)
                    self.global_force += [0,0,0,0,0,0]
                        
            if  (self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]) not in self.elcon and \
                 (self.el_dic[(x_new,y_new)],self.el_dic[(x,y)]) not in self.elcon:
                self.elcon.append([self.el_dic[(x,y)], self.el_dic[(x_new,y_new)]])
                  
                  
            if (x_new,y_new) in self.el_dic:
                if (x_new,y_new)==(1,1):
                    self.visit_list[0]+=1
                elif (x_new,y_new)==(self.m-2,1):
                    self.visit_list[1]+=1
                elif (x_new,y_new)==(self.m-2,self.n-2):
                    self.visit_list[2]+=1
                elif (x_new,y_new)==(1,self.n-2):
                    self.visit_list[3]+=1    
                                  
        return x_new, y_new
    
        
    def action_space(self,action,x0,y0):
        if action==0:
            return self.move_w(x0,y0)
        elif action==1:    
            return self.move_nw(x0,y0)
        elif action==2:  
            return self.move_n(x0,y0)
        elif action==3:
            return self.move_ne(x0,y0)
        elif action==4:
            return self.move_e(x0,y0)
        elif action==5:
            return self.move_se(x0,y0)
        elif action==6:
            return self.move_s(x0,y0)
        elif action==7:
            return self.move_sw(x0,y0)
                        
    
    def nn_input(self,x,y,action):  
        return self.line_list+[x,y]+[action]        
    
    def reward_(self,x_new,y_new,n_steps):
        reward=2*n_steps
        if all([x>=1 for x in self.visit_list]):
            reward+=10000
            weight=self.length()
        
            FEA_output_arr=self.FEA()
            max_=self.max_u(FEA_output_arr)
            strength=max_
            if weight<=self.old_weight:
                reward+=50000
                self.old_weight=weight
            if strength>=self.old_strength: 
                reward+=100000000
                self.old_strength=strength        
            self.break_flag=True     
            return reward 
        return reward     
                                   
    def draw(self,color):
        c=self.coord
        e=self.elcon
        c=np.array(c)
        e=np.array(e)
        coord=c.reshape(np.max(e)+1,3)
        fig=plt.figure(figsize=(13,5))
        for item in e:
            ax = fig.gca(projection='3d') 
            ax.plot([coord[item[0]][0],coord[item[1]][0]],\
                     [coord[item[0]][1],coord[item[1]][1]],\
                     [coord[item[0]][2],coord[item[1]][2]],
                     color=color) 
        ax.view_init(-90,90)
        ax.set_xlim([0, 5])
        ax.set_ylim([0, 5])
        plt.show()             

## Reinforcement learning model

In [15]:
N_DISCRETE_ACTIONS=8

In [16]:
class BionicEnv(gym.Env):
    
    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super().__init__()
        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        self.M=Model()
        self.M.reset(False,-500)
        self.x0=2
        self.y0=2
        self.obs=self.M.nn_input(self.x0,self.y0, 0)
        self.observation_space = spaces.Box(low=np.array([-np.inf for x in range(DIM)]),
                                            high=np.array([np.inf for y in range(DIM)]),
                                            shape=(DIM,),
                                           dtype=np.float64)
        self.step_=0
        self.needs_reset = True

    def step(self, action):
        
        x_new, y_new = self.x0, self.y0 
        x_new, y_new = self.M.action_space(action, x_new, y_new)
        self.obs=self.M.nn_input(x_new,y_new,action)
                
        self.step_+=1           
        reward=self.M.reward_(x_new,y_new,self.step_)
        self.x0,self.y0 = x_new,y_new
        
        done=False
        if self.M.break_flag:
            reward-=10000
            done=True
        
        if self.needs_reset:
            raise RuntimeError("Tried to step environment that needs reset")
            
        if done:
            self.needs_reset = True
      
        return np.array(self.obs), reward, done, dict()

    def reset(self):
        self.M.reset(False,-500)
        self.x0=2
        self.y0=2
        self.obs=self.M.nn_input(self.x0,self.y0,0)
        self.step_=0
        self.needs_reset = False
        return np.array(self.obs)  

    def render(self, mode="human"):
        self.M.draw('blue')    

    def close(self):
        pass

In [17]:
env = BionicEnv()
check_env(env)

In [18]:
start=time.time()
model = A2C("MlpPolicy", env,verbose=2).learn(total_timesteps=4_000_000)
end=time.time()   

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.56      |
|    ep_rew_mean        | 1.51e+06  |
| time/                 |           |
|    fps                | 646       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -2.03     |
|    explained_variance | 8.73e-05  |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -1.16e+04 |
|    value_loss         | 5.85e+07  |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 8.06     |
|    ep_rew_mean        | 9.91e+05 |
| time/                 |          |
|    fps                | 585      |
|    iterations         | 200      |
|

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 16.1      |
|    ep_rew_mean        | -6.98e+03 |
| time/                 |           |
|    fps                | 622       |
|    iterations         | 1400      |
|    time_elapsed       | 11        |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -1.2      |
|    explained_variance | 0.000719  |
|    learning_rate      | 0.0007    |
|    n_updates          | 1399      |
|    policy_loss        | 34.2      |
|    value_loss         | 4.56e+03  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 16.4      |
|    ep_rew_mean        | -6.07e+03 |
| time/                 |           |
|    fps                | 623       |
|    iterations         | 1500      |
|    time_elapsed       | 12        |
|    total_timesteps    | 7500      |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 18.6      |
|    ep_rew_mean        | -7.79e+03 |
| time/                 |           |
|    fps                | 612       |
|    iterations         | 2700      |
|    time_elapsed       | 22        |
|    total_timesteps    | 13500     |
| train/                |           |
|    entropy_loss       | -0.669    |
|    explained_variance | 0.000309  |
|    learning_rate      | 0.0007    |
|    n_updates          | 2699      |
|    policy_loss        | 30.2      |
|    value_loss         | 6.31e+03  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 19.5      |
|    ep_rew_mean        | -7.96e+03 |
| time/                 |           |
|    fps                | 605       |
|    iterations         | 2800      |
|    time_elapsed       | 23        |
|    total_timesteps    | 14000     |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 20.3      |
|    ep_rew_mean        | -5.53e+03 |
| time/                 |           |
|    fps                | 578       |
|    iterations         | 4000      |
|    time_elapsed       | 34        |
|    total_timesteps    | 20000     |
| train/                |           |
|    entropy_loss       | -0.644    |
|    explained_variance | -1.29e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 3999      |
|    policy_loss        | 10.6      |
|    value_loss         | 1.53e+03  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 20.4      |
|    ep_rew_mean        | -6.23e+03 |
| time/                 |           |
|    fps                | 580       |
|    iterations         | 4100      |
|    time_elapsed       | 35        |
|    total_timesteps    | 20500     |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 20.8      |
|    ep_rew_mean        | -9.22e+03 |
| time/                 |           |
|    fps                | 595       |
|    iterations         | 5300      |
|    time_elapsed       | 44        |
|    total_timesteps    | 26500     |
| train/                |           |
|    entropy_loss       | -0.615    |
|    explained_variance | 0.0013    |
|    learning_rate      | 0.0007    |
|    n_updates          | 5299      |
|    policy_loss        | 81.6      |
|    value_loss         | 1.33e+04  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 20.6      |
|    ep_rew_mean        | -8.93e+03 |
| time/                 |           |
|    fps                | 596       |
|    iterations         | 5400      |
|    time_elapsed       | 45        |
|    total_timesteps    | 27000     |
| train/    

...

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 40        |
|    ep_rew_mean        | 1.64e+03  |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 144400    |
|    time_elapsed       | 1733      |
|    total_timesteps    | 722000    |
| train/                |           |
|    entropy_loss       | -0.000861 |
|    explained_variance | 0.999     |
|    learning_rate      | 0.0007    |
|    n_updates          | 144399    |
|    policy_loss        | 0.000824  |
|    value_loss         | 60.5      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 40        |
|    ep_rew_mean        | 1.64e+03  |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 144500    |
|    time_elapsed       | 1735      |
|    total_timesteps    | 722500    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 34.8      |
|    ep_rew_mean        | -8.75e+03 |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 145700    |
|    time_elapsed       | 1749      |
|    total_timesteps    | 728500    |
| train/                |           |
|    entropy_loss       | -0.00013  |
|    explained_variance | 0.267     |
|    learning_rate      | 0.0007    |
|    n_updates          | 145699    |
|    policy_loss        | -0.00936  |
|    value_loss         | 1.4e+07   |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 34.8      |
|    ep_rew_mean        | -8.75e+03 |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 145800    |
|    time_elapsed       | 1750      |
|    total_timesteps    | 729000    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 34.9      |
|    ep_rew_mean        | -8.75e+03 |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 147000    |
|    time_elapsed       | 1762      |
|    total_timesteps    | 735000    |
| train/                |           |
|    entropy_loss       | -0.00736  |
|    explained_variance | 0.991     |
|    learning_rate      | 0.0007    |
|    n_updates          | 146999    |
|    policy_loss        | -0.00407  |
|    value_loss         | 24.2      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 34.9      |
|    ep_rew_mean        | -8.75e+03 |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 147100    |
|    time_elapsed       | 1763      |
|    total_timesteps    | 735500    |
| train/    

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 29.6      |
|    ep_rew_mean        | -9.04e+03 |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 148300    |
|    time_elapsed       | 1780      |
|    total_timesteps    | 741500    |
| train/                |           |
|    entropy_loss       | -0.00193  |
|    explained_variance | 0.721     |
|    learning_rate      | 0.0007    |
|    n_updates          | 148299    |
|    policy_loss        | -0.0145   |
|    value_loss         | 2.64e+03  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 25.8      |
|    ep_rew_mean        | -9.25e+03 |
| time/                 |           |
|    fps                | 416       |
|    iterations         | 148400    |
|    time_elapsed       | 1781      |
|    total_timesteps    | 742000    |
| train/    

KeyboardInterrupt: 

In [ ]:
print('Total time taken: {} min'.format((end - start)/60))

### Design by AI

In [ ]:
obs = env.reset()

In [ ]:
i=0
while i<100:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    if env.M.break_flag:
        break
    i+=1   

In [ ]:
print(i)

In [ ]:
env.render() 

In [ ]:
env.M.length()

In [ ]:
FEA_output_arr=env.M.FEA()
env.M.max_u(FEA_output_arr)